In [139]:
import pandas as pd 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_recall_fscore_support

In [22]:
metadata = pd.read_csv('movies.csv', low_memory=False)
metadata_rating = pd.read_csv('ratings.csv', low_memory=False)
metadata.head(3)

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance


In [90]:
tfidf = TfidfVectorizer(stop_words='english')
metadata['title'] = metadata['title'].fillna('')
tfidf_matrix = tfidf.fit_transform(metadata['title'])
movie_name=['Toy Story']
movie_tfidf = tfidf.transform(movie_name)

In [126]:
n_neighbors = 11
KNN = NearestNeighbors(n_neighbors,p=2)
KNN.fit(tfidf_matrix)
NNs = KNN.kneighbors(movie_tfidf,return_distance=True)

In [127]:
def get_recommendation(top, df_all, scores):
  recommendation = pd.DataFrame(columns = [ 'movieId', 'title'])
  count = 0
  for i in top:
      recommendation.at[count, 'movieId'] = df_all['movieId'][i]
      recommendation.at[count, 'title'] = df_all['title'][i]
      recommendation.at[count, 'score'] =  scores[count]
      count += 1
  return recommendation

In [128]:
top = NNs[1][0][1:]
index_score = NNs[0][0][1:]
df_all = metadata[['movieId','title']]
recommendation_knn=get_recommendation(top,df_all,index_score)
recommendation_knn

,movieId,title,score
0,3114,Toy Story 2 (1999),0.456561
1,1,Toy Story (1995),0.468064
2,106022,Toy Story of Terror (2013),0.668171
3,4929,"Toy, The (1982)",0.770886
4,120474,Toy Story That Time Forgot (2014),0.849089
5,80141,"Christmas Toy, The (1986)",0.887103
6,122078,The Toy Wife (1938),0.936509
7,159856,The Toy Box (1971),0.939533
8,143537,Toy Masters (2014),0.941782
9,5843,Toy Soldiers (1991),0.956857
